### Generating a toy-case topology

In [1]:
import numpy as np 
import cvxpy as cp

from itertools import combinations
from tqdm import tqdm

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [3]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [4]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [5]:
Y = U @ np.diag(H) @ U.T @ X

#Y += np.random.normal(0, 10e-2, size=Y.shape)

In [6]:
np.trace(X.T @ L_f @ X)

13638.720275074294

In [7]:
np.trace(Y.T @ L_f @ Y)

9.565305954913251

In [8]:
for edge in edges:
    u = edge[0]
    v = edge[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]

    Fu = F[edge][u]
    Fv = F[edge][v]
    print(np.linalg.norm(X_u - Fu @ X_u), np.linalg.norm(X_u))
    print(edge, edge in edges, np.linalg.norm(Fu @ X_u - Fv @ X_v ))

6.983802450991694 4.206960829585515
(0, 1) True 1.440294217831587
5.764450607841412 4.206960829585515
(0, 3) True 1.0180201266978748
4.373151743018024 4.206960829585515
(0, 6) True 1.1647711091415438
5.659202709335148 5.297308929003016
(1, 2) True 1.0125850945767727
4.5853387089971935 5.297308929003016
(1, 5) True 0.8138985625980523
2.256325480657027 2.009558616535624
(2, 4) True 1.037551548707249
1.445910079993538 1.6028552084701946
(4, 6) True 1.0720146981535497
4.384806776371905 3.0054478726483564
(5, 6) True 1.0882617482486012


____________

In [10]:
def premultiplier(Xu, Xv, d, beta = 1e-3):
    A11 = np.kron((Xu @ Xu.T), np.eye(d)) + beta * np.eye(d**2)
    A12 = - np.kron((Xu @ Xv.T), np.eye(d))
    A21 = - np.kron((Xv @ Xu.T), np.eye(d))
    A22 = np.kron((Xv @ Xv.T), np.eye(d)) + beta * np.eye(d**2)

    return np.block([[A11, A12], [A21, A22]])

def solver(Xu, Xv, d): 
    A = premultiplier(Xu, Xv, d)
    b = np.block([[np.eye(d).flatten().reshape(-1,1)],[np.eye(d).flatten().reshape(-1,1)]])

    sol = np.linalg.solve(A,b)

    Fu = sol[0:d*d,:]
    Fv = sol[d*d:,:]

    return Fu.reshape(d,d), Fv.reshape(d,d)

def KKT_solver(V, mu, Y, d):
    T = 0

    H = {
        edge : {
            edge[0] : None,
            edge[1] : None
        }
    for edge in combinations(range(V), 2)
    }

    for e in combinations(range(V),2):
        u = e[0]
        v = e[1]

        X_u = Y[u*d:(u+1)*d,:]
        X_v = Y[v*d:(v+1)*d,:]

        Fu, Fv = solver(X_u, X_v, d)

        H[e][u] = Fu
        H[e][v] = Fv
        
        T += np.trace(H[e][u]) + np.trace(H[e][v])

    F = {
        edge : {
            edge[0] : - mu/T * (H[edge][edge[0]]),
            edge[1] : - mu/T * (H[edge][edge[1]])
        }
    for edge in combinations(range(V), 2)
    }
    
    return F

In [11]:
maps_ = KKT_solver(V, 70, Y, 3)

In [12]:
B_hat_2 = np.zeros((d*E, d*V))

for i, edge in enumerate(edges):

    u = edge[0] 
    v = edge[1] 

    B_u = maps_[edge][u]
    B_v = maps_[edge][v]

    B_hat_2[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat_2[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat_2 = B_hat_2.T @ B_hat_2

In [13]:
np.trace(Y.T @ L_f_hat_2 @ Y)

280.9230791541997

In [14]:
E0 = len(edges)
all_edges = list(combinations(range(V), 2))

energies = {
    e : 0
    for e in all_edges
    }

for e in all_edges:
    X_u = Y[d*e[0]:(e[0]+1)*d,:]
    X_v = Y[d*e[1]:(e[1]+1)*d,:]

    energies[e] = np.linalg.norm(maps_[e][e[0]] @ X_u - maps_[e][e[1]] @ X_v)

retrieved = sorted(energies.items(), key=lambda x:x[1])

acc = len(set(list(map(lambda x: x[0], retrieved[:E0]))).intersection(set(edges[:E0]))) / E0

edges_subset = list(set(list(map(lambda x: x[0], retrieved[:E0]))).intersection(set(edges[:E0])))

In [15]:
retrieved

[((2, 5), 0.11739848185247624),
 ((4, 5), 0.2450092898153875),
 ((2, 3), 0.39750611496625743),
 ((2, 4), 0.5290603687203558),
 ((0, 2), 0.5619913176999537),
 ((3, 4), 0.6502483122428856),
 ((0, 1), 0.6558713003723936),
 ((4, 6), 0.6563954704868576),
 ((1, 3), 0.8410992503684015),
 ((1, 4), 1.2508618580372362),
 ((2, 6), 1.356615160289579),
 ((0, 4), 1.5142178131288502),
 ((1, 2), 1.5693118437756368),
 ((3, 5), 1.8298238785406236),
 ((3, 6), 1.915886157046632),
 ((0, 5), 2.6490303678417755),
 ((1, 6), 2.899509225682434),
 ((0, 6), 3.539704264427376),
 ((5, 6), 3.8362261019818225),
 ((1, 5), 4.736969232044972),
 ((0, 3), 15.087557719877468)]

In [16]:
F[(0,1)]

{0: array([[ 0.4049829 ,  0.37917453, -1.57785554],
        [-0.29087592,  1.44032104,  0.5504921 ],
        [ 0.24729945, -0.101183  , -0.49140756]]),
 1: array([[-0.04367096, -1.02449401,  0.20367415],
        [-0.16662979, -0.76331149, -0.07396818],
        [ 1.10366836, -0.45751093, -0.5156448 ]])}

____________

In [17]:
class SheafSolver():
    def __init__(
            self, 
            d, 
            V, 
            mu, 
            gamma, 
            alpha, 
            beta, 
            Y):

        # Sheaf structure
        self.d = d
        self.V = V

        # ADMM hyperparameters
        self.mu = mu
        self.gamma = gamma

        # Sparsification hyperparameters
        self.alpha = alpha
        self.beta = beta

        # Observsed signals
        self.Y = Y
        
        # Initialization
        self.F = {
            edge: {
                edge[0]: None,
                edge[1]: None
            } for edge in combinations(range(self.V),2)
        }

        # Setup
        self.BlockMatricesPrecomp()

    def BlockMatricesPrecomp(self):
        self.A = {edge:None for edge in combinations(range(self.V),2)}
        for edge in tqdm(combinations(range(self.V),2)):
            u = edge[0]
            v = edge[1]

            Yu = self.Y[u*self.d:(u+1)*self.d,:]
            Yv = self.Y[v*self.d:(v+1)*self.d,:]

            A11 = np.kron(Yu @ Yu.T, np.eye(self.d)) + 1e-3*np.eye(self.d**2)
            A12 = - np.kron(Yu @ Yv.T, np.eye(self.d)) + self.gamma*np.eye(self.d**2)
            A21 = - np.kron(Yv @ Yu.T, np.eye(self.d)) + self.gamma*np.eye(self.d**2)
            A22 = np.kron(Yv @ Yv.T, np.eye(self.d)) + 1e-3 *np.eye(self.d**2)

            A = np.block([[A11, A12],[A21, A22]])
            self.A[edge] = np.linalg.inv(A)

    # Phase 1 - Optimize the restriction maps

    def MapsOptimization(self):
        for edge in combinations(range(self.V),2):
            b = np.block([[- self.mu *np.eye(self.d).flatten().reshape(-1,1)],
                          [- self.mu *np.eye(self.d).flatten().reshape(-1,1)]])

            sol = np.linalg.solve(self.A[edge],b)

            self.F[edge][edge[0]] = sol[0:self.d*self.d,:].reshape(self.d,self.d)
            self.F[edge][edge[1]] = sol[self.d*self.d:,:].reshape(self.d,self.d)

    # Phase 2 - Sparsify the graph (Kalofolias)

    def PairwiseDistances(self):
        self.Z = np.zeros((self.V,self.V))
        for u in range(self.V):
            for v in range(u+1, self.V):
                Yu = self.Y[u*self.d:(u+1)*self.d,:]
                Yv = self.Y[v*self.d:(v+1)*self.d,:]

                dist = np.linalg.norm(self.F[(u,v)][u] @ Yu - self.F[(u,v)][v] @ Yv)

                self.Z[u,v] = dist
                self.Z[v,u] = dist

    def KalofoliasOptimization(self):
        # Define the variable
        W = cp.Variable((self.V, self.V), symmetric=True)

        # Constraints: diagonal elements must be zero
        constraints = [cp.diag(W) == 0]

        # Objective function components
        term1 = cp.norm1(cp.multiply(W, self.Z))  # ||W ∘ Z||_1,1
        term2 = - self.alpha * cp.sum(cp.log(cp.sum(W, axis=1)))  # -α * 1^T log(W1)
        term3 = (self.beta / 2) * cp.norm(W, 'fro')**2  # (β/2) * ||W||_F^2

        # Define the objective
        objective = cp.Minimize(term1 + term2 + term3)

        # Define the problem
        problem = cp.Problem(objective, constraints)

        # Solve the problem
        problem.solve()

        # Return the optimal solution
        self.W = W.value
        
    def EdgeRetrieval(self):
        for u in range(self.V):
            for v in range(u+1,V):
                if self.W[u,v] < 1e-5:
                    self.W[u,v] = 0
                    self.W[v,u] = 0
        retrieved_edges = []

        for u in range(V):
            for v in range(u,V):
                if self.W[u,v] != 0:
                    retrieved_edges.append((u,v))

        self.retrieved_edges = retrieved_edges
    
    def MapsCorrection(self):
        for u in range(self.V):
            for v in range(u+1,V):
                self.F[(u,v)][u] *= self.W[u,v]**2
                self.F[(u,v)][v] *= self.W[u,v]**2 

    def Learning(self):
        self.MapsOptimization()
        self.PairwiseDistances()
        self.KalofoliasOptimization()
        self.EdgeRetrieval()
        self.MapsCorrection()

In [18]:
# Let's generate a toy topology for our example

V = 100
nodes = np.arange(V)

# Probability p = 1.1 * log(V) / V
p = 1.1 * np.log(V) / V

edges = []
for u in range(V):
    for v in range(u+1, V):
        s = np.random.rand()
        if s < p:
            edges.append((u,v))


d = 10                                           # Node and edges stalks dimension
E = len(edges)
F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

In [19]:
N = 100
X = np.random.randn(V*d,N)

In [20]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [21]:
Y = U @ np.diag(H) @ U.T @ X

In [22]:
solver = SheafSolver(d, V, 1, 10, 3, 5, Y)

0it [00:00, ?it/s]

4950it [00:43, 112.61it/s]


In [23]:
solver.Learning()

In [24]:
len(set(solver.retrieved_edges).intersection(set(edges)))/len(edges)

0.92578125

In [25]:
len(set(solver.retrieved_edges).intersection(set(edges)))/len(solver.retrieved_edges)

0.0602287166454892